## 1. Carrega as bases de dados

In [1]:
import pandas as pd

# Modelo
#MODELO = 'text-embedding-3-large'
#MODELO = 'text-embedding-3-small'
MODELO = 'text-embedding-ada-002'

# Pasta com os dados
PASTA_DADOS = './dados/'

# Pasta com os dados de jurisprudência já tratados
PASTA_JURIS_TCU = f'{PASTA_DADOS}outputs/1_tratamento_juris_tcu/'

# Pasta onde serão armazenados os resultados desse caderno
PASTA_RESULTADO_CADERNO = f'{PASTA_DADOS}outputs/7_gera_embeddings_termos_open_ai/{MODELO}/'

# Substituir embeddings já criados
SOBRESCREVER_EMBEDDINGS = False

# Tamanho do lote
TAMANHO_DO_LOTE = 150

# Carrega os arquivos 
def carrega_juris_tcu():
    doc1 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_1.csv', sep='|')
    doc2 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_2.csv', sep='|')
    doc3 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_3.csv', sep='|')
    doc4 = pd.read_csv(f'{PASTA_JURIS_TCU}doc_tratado_parte_4.csv', sep='|')
    doc = pd.concat([doc1, doc2, doc3, doc4], ignore_index=True)
    query = pd.read_csv(f'{PASTA_JURIS_TCU}query_tratado.csv', sep='|')
    qrel = pd.read_csv(f'{PASTA_JURIS_TCU}qrel_tratado.csv', sep='|')

    return doc, query, qrel

In [2]:
from formatador import remove_html

# Carrega as queries para query
doc, query, qrel = carrega_juris_tcu()

#Transforma dataframe em dicionário
query = query.to_dict(orient='list')
print(query.keys())

dict_keys(['KEY', 'TEXT', 'SOURCE'])


## 2. Obtenção dos embeddings.

In [3]:
# Função para dividir dicionário em lotes
def dividir_dicionario_em_lotes(dicionario, tamanho_do_lote):
    vetor_de_dicionarios = []
    max_len = max(len(v) for v in dicionario.values())  # Encontrando o vetor de valores mais longo
    
    for i in range(0, max_len, tamanho_do_lote):
        novo_dicionario = {chave: valores[i:i + tamanho_do_lote] for chave, valores in dicionario.items()}
        vetor_de_dicionarios.append(novo_dicionario)
    
    return vetor_de_dicionarios

In [4]:
import numpy as np
import torch

# Função para reconstruir dicionário a partir de lotes
def reconstruir_dicionario_a_partir_de_lotes(vetor_de_dicionarios):
    dicionario_reconstruido = {}
    
    # Inicializando listas vazias para cada chave no primeiro dicionário do vetor
    for chave in vetor_de_dicionarios[0].keys():
        dicionario_reconstruido[chave] = []
    
    # Iterando sobre cada dicionário no vetor e concatenando os valores para cada chave
    for dicionario_lote in vetor_de_dicionarios:
        for chave, valores in dicionario_lote.items():
            dicionario_reconstruido[chave].extend(valores)
    
    # Transforma numpy array em tensor
    dicionario_reconstruido['mean_hidden_state'] = torch.tensor(np.array(dicionario_reconstruido['mean_hidden_state']))
    
    return dicionario_reconstruido

In [5]:
# Função para converter dicionário em dataframe
def dicionario_para_dataframe(dicionario):
    df = pd.DataFrame(dicionario)
    return df

In [6]:
# Função para converter dataframe em dicionario
def dataframe_para_dicionario(df):
    return df.to_dict(orient='list')

In [7]:
import openai
from getpass import getpass

openai.api_key = getpass('Qual a chave da OpenAI?')

# Função para extrair embeddings
def get_embedding(texto, model=MODELO):
    texto = texto.replace("\n", " ")
    return openai.embeddings.create(input = [texto], model=model).data[0].embedding

Qual a chave da OpenAI?········


In [8]:
# Processar e salvar embeddings
import pickle
from tqdm import tqdm
import os

# Divide query em lotes
query_em_lotes = dividir_dicionario_em_lotes(query, TAMANHO_DO_LOTE)

# Processa e salva embeddings
for i, dicionario in enumerate(tqdm(query_em_lotes), start=1):
    
    caminho_arquivo = f'{PASTA_RESULTADO_CADERNO}{MODELO}_embeddings_query_{i}.pickle'
    if  not SOBRESCREVER_EMBEDDINGS and os.path.exists(caminho_arquivo):
        continue

    dicionario_df = dicionario_para_dataframe(dicionario) 
    dicionario_df['mean_hidden_state'] = dicionario_df.TEXT.apply(lambda x: get_embedding(x, model=MODELO))
    
    # Cria estrutura que será salva em arquivo
    embeddings_js = {
        'key': dicionario_df['KEY'].tolist(),
        'mean_hidden_state': dicionario_df['mean_hidden_state'].tolist()
    }
    
    # Gravando lote em um arquivo .pickle
    with open(caminho_arquivo, 'wb') as arquivo_pickle:
        pickle.dump(embeddings_js, arquivo_pickle)

100%|██████████| 1/1 [00:45<00:00, 45.03s/it]


## 4. Cálculo da distância entre os embeddings.

In [9]:
# Função para restaurar embeddings dos arquivos pickle

import os
import pickle

def restaurar_query_encoded_de_pickle(pasta_resultado_caderno):
    # Lista para armazenar os dicionários lidos dos arquivos .pickle
    query_encoded_restaurado = []

    # Listando todos os arquivos .pickle no diretório especificado
    arquivos_pickle = [arq for arq in os.listdir(pasta_resultado_caderno) if arq.endswith('.pickle')]

    # Ordenando os arquivos pelo número (assumindo que os nomes dos arquivos seguem o padrão embeddings_query_X.pickle)
    arquivos_pickle.sort(key=lambda x: int(x.split('_')[-1].split('.')[0]))

    # Lendo cada arquivo .pickle e restaurando o dicionário
    for nome_arquivo in arquivos_pickle:
        caminho_arquivo = os.path.join(pasta_resultado_caderno, nome_arquivo)
        with open(caminho_arquivo, 'rb') as arquivo_pickle:
            dicionario_restaurado = pickle.load(arquivo_pickle)
            query_encoded_restaurado.append(dicionario_restaurado)

    return reconstruir_dicionario_a_partir_de_lotes(query_encoded_restaurado)

In [10]:
query_encoded_restaurado = restaurar_query_encoded_de_pickle(PASTA_RESULTADO_CADERNO)

In [11]:
import numpy as np

query_hidden = query.copy()
query_hidden['mean_hidden_state'] = query_encoded_restaurado['mean_hidden_state']
print(f"mean_hidden_state: {query_hidden['mean_hidden_state'].size()}")

mean_hidden_state: torch.Size([150, 1536])


In [12]:
# Extraindo os embeddings de duas queries
embedding1_tensor = query_hidden['mean_hidden_state'][0]
embedding2_tensor = query_hidden['mean_hidden_state'][4]

# Normalizando os embeddings
embedding1_norm = embedding1_tensor / embedding1_tensor.norm()
embedding2_norm = embedding2_tensor / embedding2_tensor.norm()

# Calculando a similaridade por cosseno
cosine_similarity = torch.dot(embedding1_norm, embedding2_norm)

print(f"Query 1: {query_hidden['TEXT'][0]}")
print(f"Query 2: {query_hidden['TEXT'][4]}")
print(f"Similaridade por cosseno: {cosine_similarity.item()}")

Query 1: técnica e preço
Query 2: sobrepreço e superfaturamento
Similaridade por cosseno: 0.8464457046137243


In [13]:
query_encoded_restaurado

{'key': [1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150],
 'mean_hidden_state': tensor([[-0.0128,  0.0040,  

In [14]:
print(query_hidden['TEXT'][0])
print(query_hidden['TEXT'][1])
print(query_hidden['TEXT'][4])

técnica e preço
restos a pagar
sobrepreço e superfaturamento
